Install the auto_echem package. You need git installed onto your computer (https://git-scm.com/download/win)

In [1]:
pip install "git+https://github.com/lfo96/auto_echem.git"  

  Cloning https://github.com/lfo96/auto_echem.git to c:\users\loren\appdata\local\temp\pip-req-build-lv25x3eu
  Resolved https://github.com/lfo96/auto_echem.git to commit db5da2556b0e33df6aacf8e8bec7651a27988153
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/lfo96/auto_echem.git 'C:\Users\loren\AppData\Local\Temp\pip-req-build-lv25x3eu'


This needs to be installed only once. Can be removed from the notebook thereafter.  
I reguarly update the functions, so if you want to have the latest version on here either keep your local respitory udpated or run the pip update from time to time.

In [6]:
pip install -U "git+https://github.com/lfo96/auto_echem.git

  Cloning https://github.com/lfo96/auto_echem.git to c:\users\quee4145\appdata\local\temp\pip-req-build-53oashld
  Created wheel for Auto-EChem: filename=Auto_EChem-0.0.1-py3-none-any.whl size=32025 sha256=74c539a4bbf203a684b342c5d01393327446bbf7ad1880944d9892eefd1a57c6
  Stored in directory: C:\Users\quee4145\AppData\Local\Temp\pip-ephem-wheel-cache-q8hgck7g\wheels\1b\09\6f\cfe715c32b6d1bb1aaa5b9efc2a31198c695ef340796335a68
Successfully built Auto-EChem
  Attempting uninstall: Auto-EChem
    Found existing installation: Auto-EChem 0.0.1
    Uninstalling Auto-EChem-0.0.1:
      Successfully uninstalled Auto-EChem-0.0.1
Note: you may need to restart the kernel to use updated packages.


Create notebook in a directionary where you also want to store your plots. I usually create a new notebook for every new set of measurements I want to evaluate.  
Import the functions from auto_echem into your notebook:

In [7]:
# Those are the functions you need 
from auto_echem.GCPL_functions import *
from auto_echem.general_functions import *
from auto_echem.three_el_functions import *
from auto_echem.auto import *
from auto_echem.impedance_functions import *
from auto_echem.TGA_DSC_functions import *
from auto_echem.XPS_functions import *

Next I specify where my biologic data is stored. Conveniently you can map the mp_pastagroup network drive where the data is automatically updated in real time (or whenever backup360 creates its backups...)  
Always specify the lab setting file created by ec-lab. It has the .mps format. You can use the test files in "auto_echem\testfiles\20221028_FeF2_bm_R52_US_801307_CCCC_1MLiTFSIPyr13TFSI_1\20221028_FeF2_bm_R52_US_801307_CCCC_1MLiTFSIPyr13TFSI_1.mps" to try and run this notebook.

note: use r' before your pathway to make sure python can read it properly

In [1]:
test_file_p = r"packagelocation\auto_echem\testfiles\20221028_FeF2_bm_R52_US_801307_CCCC_1MLiTFSIPyr13TFSI_1\20221028_FeF2_bm_R52_US_801307_CCCC_1MLiTFSIPyr13TFSI_1.mps"

From here your analysis is dependent on your measurement file.  
auto_echem functions can read in and process various measurements file generated by EC-lab (and BT-lab), but also TGA-DSC, XRD, XPS.
\
\
The most generic analysis uses the "auto" function. The function reads in the .mps file, extract the specified measurements and subsequently look for through the folder of the setting file for the respesctive .mpr files. Each .mpr file is read in and evaluated respectivly.


In [ ]:
test_file = auto(test_file_p, circ = ['RpC'])

Usually I let the auto function run over my measurement and continue from there on. The auto function returns a dictionary (dict) with meta data, all the raw data (dict['data']), and the evaluated raw data (dict['eva']).

In this example, we analyzed a measurement of a coin cell, starting with a resting period measuring the OCV, PEIS, and looping 23 times. Subsequently the GCPL is measured. 

The auto function has detected and analyzed OCV, PEIS and GCPL in the measurement folder.
The dictionary contains amongst other, the following information: 

In [ ]:

print('Your active material mass is (mg):',test_file['active material mass'])
print('Your electrode surface area is (cm2):',test_file['electrode surface area'])
print('Your test protocol is:',test_file['protocol'])

Your raw data is saved into the dictionary and can be accessed by dict['data']. Therein each measurement gets its own entry with a number indicating the chronology of the measurement. For example:

In [ ]:
for meas_data in test_file['data']:
    print(meas_data)

Similiarily, the raw data (stored in ['data']) is processed, evaluated, and stored in dict['eva'].

In [ ]:
for eva_data in test_file['eva']:
    print(eva_data)

Since there is not much to analzye for OCV measurement, there is no entry in the 'eva' section.  
The "1 PEIS" contains various information including for example each parameter order by cycle.


In [ ]:
test_file['eva']['1 PEIS']['Nyquist parameter']

Error Shooting Time

In [16]:
test = r"Y:\VMP3 datafiles\Giulia\PbPbF2-PTFE-CF-Guillaume\0.5m TMAF(10m) MeOH PN\10102022-PbPbF2symm-05mTMAF-10mMeOH-PN-PTFE-CF-30um-n1.mps"

In [19]:
filename = test
    #obtain number of header lines in the text file
with open(filename,encoding= 'ISO-8859-1') as fin:
    head_len = manual_header(filename)   
    t = 0
    techniques = []
    m_am = A_el = cd = t_wait= np.nan
# head lenght in the beginning of file does not count itself and thus needs to be shifted for 4 rows.
    for line in islice(fin, 0,head_len):
        #print(line.split(":")[0])
        if line.split(":")[0]=="Mass of active material ": #extract the active material mass
            m_am = float(line.split(":")[1].split(" ")[1]) 
        elif line.split(":")[0]=="Electrode surface area ": #extract the electrode surface area
            A_el = float(line.split(":")[1].split(" ")[1]) 
            if A_el == 0.001:
                print('Electrode Surface Area manually set to 1.13cm2.')
                A_el = 1.13097
        elif line.split(":")[0]=="Cycle Definition ": #extract the electrode surface area
            cd = line.split(":")[1].split(" ")[1] 
        elif t == 1:  
            technique = line.split("\n")[0]
            if technique == 'Modulo Bat':
                t=0
                print('Modulo Bat found.')
                continue
            try:
                techniques.append(tech[technique])
                t = 0
            except KeyError:
                print('Unknown technique found: '+str(technique))
                t = 0
        elif line.split(":")[0]=="Technique ":
            t = 1
        elif line.split(":")[0]== 'td (h':
            if np.isnan(t_wait) == False:
                print('Several waiting times found.')
                t_wait_2 = datetime(line.split(" ")[11])
            t_wait = datetime(line.split(" ")[11])
        elif line.split(":")[0]== 'tR (h':
            # GCPL and OCV both have a tR entry in the meta file. Necessary to distinguish both
            if techniques[-1] == 'GCPL':
                t_GCPL = []
                for entry in line.split('        '):
                    try:
                        t_GCPL.append(datetime(entry.strip()))
                    except ValueError:
                        pass
                
            else:
                if np.isnan(t_wait) == False:
                    print('Several waiting times found.')
                    t_wait_2 = datetime(line.split(" ")[11])
                t_wait = datetime(line.split(" ")[11])
        elif line[0:9]=='ctrl_type':
            # Add all the techniques from the MB setting to techniques list.
            MB_tech = line[9:].split(' ')
            for t_i in MB_tech[:-2]:
                if t_i != '':
                    techniques.append(t_i)

#create a dictionary with all the important meta data
meta_data = {
    'header': head_len,
    'active material mass': m_am,
    'electrode surface area': A_el,
    'protocol': techniques,
    'cd' : cd,
    'waiting time' : t_wait
}
try:
        meta_data['waiting time 2'] = t_wait_2
except NameError:
    pass  
try:
        meta_data['t_GCPL'] = t_GCPL
except NameError:
    pass             

return(meta_data)

Electrode Surface Area manually set to 1.13cm2.


ValueError: time data '24:00:10' does not match format '%H:%M:%S'

In [39]:
time = '24:00:10'

In [40]:
start_dt = dt.datetime.strptime("00:00:00", '%H:%M:%S')
time_float = float('{:0.3f}'.format((dt.datetime.strptime(time, '%H:%M:%S') - start_dt).seconds/3600))

ValueError: time data '24:00:10' does not match format '%H:%M:%S'

In [42]:
def convert_time(time_str):
    try:
        dt_timeform = dt.datetime.strptime(time, '%H:%M:%S')
    except ValueError:
        h_remain = int(time.split(':')[0])-23
        
    return(dt_timeform)

1

In [35]:
start_dt

datetime.datetime(1900, 1, 1, 0, 0)

In [26]:
dt.datetime.strptime('24:00:10', '%H:%M:%S')

ValueError: time data '24:00:10' does not match format '%H:%M:%S'